In [1]:
import torch

In [2]:
cos = torch.load('cos')
sin = torch.load('sin')
rotary_pos_emb_causal = torch.load('rotary_pos_emb_causal')


In [3]:
# rotary_pos_emb_flashinfer = torch.cat((cos, sin), dim=1).view(7, 32, 2)
# rotary_pos_emb_flashinfer = torch.cat((cos.squeeze(2), sin.squeeze(2)), dim=2)
rotary_pos_emb_flashinfer = torch.cat((cos.transpose(1, 2), sin.transpose(1, 2)), dim=2)
torch.all(rotary_pos_emb_flashinfer == rotary_pos_emb_causal[0])

tensor(True, device='cuda:0')

In [7]:
key_layer_before_causal = torch.load("key_layer_before_causal")
query_layer_before_causal = torch.load("query_layer_before_causal")
key_layer_before_flashinfer = torch.load("key_layer_before_flashinfer")
query_layer_before_flashinfer = torch.load("query_layer_before_flashinfer")
key_layer_after_causal = torch.load("key_layer_after_causal")
query_layer_after_causal = torch.load("query_layer_after_causal")
key_layer_after_flashinfer = torch.load("key_layer_after_flashinfer")
query_layer_after_flashinfer = torch.load("query_layer_after_flashinfer")


In [22]:
query_layer_before_flashinfer.shape, rotary_pos_emb_flashinfer.shape

(torch.Size([7, 32, 128]), torch.Size([7, 32, 2]))

In [26]:
res = apply_rotary_pos_emb(key_layer_before_flashinfer.transpose(0, 1).unsqueeze(0), rotary_pos_emb_flashinfer.unsqueeze(0))
assert_close(res, key_layer_after_causal, 1e-2, 3e-3)
res.shape

torch.Size([1, 2, 7, 128])

In [5]:
def apply_rotary_pos_emb(x: torch.Tensor, rope_cache: torch.Tensor) -> torch.Tensor:
    # x: [b, np, sq, hn]
    b, np, sq, hn = x.size(0), x.size(1), x.size(2), x.size(3)
    rot_dim = rope_cache.shape[-2] * 2
    x, x_pass = x[..., :rot_dim], x[..., rot_dim:]
    # truncate to support variable sizes
    rope_cache = rope_cache[:, :sq]
    xshaped = x.reshape(b, np, sq, rot_dim // 2, 2)
    rope_cache = rope_cache.view(-1, 1, sq, xshaped.size(3), 2)
    x_out2 = torch.stack(
        [
            xshaped[..., 0] * rope_cache[..., 0] - xshaped[..., 1] * rope_cache[..., 1],
            xshaped[..., 1] * rope_cache[..., 0] + xshaped[..., 0] * rope_cache[..., 1],
        ],
        -1,
    )
    x_out2 = x_out2.flatten(3)
    return torch.cat((x_out2, x_pass), dim=-1)

In [12]:
key_after_rope = apply_rotary_pos_emb(key_layer_before_causal, rotary_pos_emb_causal)

In [13]:
x = key_layer_before_causal
rope_cache = rotary_pos_emb_causal

In [14]:
b, np, sq, hn = x.size(0), x.size(1), x.size(2), x.size(3)
rot_dim = rope_cache.shape[-2] * 2
x, x_pass = x[..., :rot_dim], x[..., rot_dim:]
# truncate to support variable sizes
rope_cache = rope_cache[:, :sq]
xshaped = x.reshape(b, np, sq, rot_dim // 2, 2)

In [16]:
rope_cache = rope_cache.view(-1, 1, sq, xshaped.size(3), 2)

In [17]:
x1 = xshaped[..., 0] * rope_cache[..., 0] - xshaped[..., 1] * rope_cache[..., 1]
x2 = xshaped[..., 1] * rope_cache[..., 0] + xshaped[..., 0] * rope_cache[..., 1]

In [19]:
x1.shape, rot_dim

(torch.Size([1, 2, 7, 32]), 64)

In [9]:
def assert_close(a, b, rtol, atol):
    # rtol, atol = {
    #     torch.float16: (1e-2, 5e-2),
    #     torch.bfloat16: (8e-3, 8e-3),
    # }[a.dtype]
    torch.testing.assert_close(a, b, rtol=rtol, atol=atol)